# Objective

#### We want to export Google trends to a csv for a list of (predefined) keywords. 
#### => Ultimately, we'll have data for when each keyword was searched most over a (given) period of time. 

 

# Step 1: Create a keyword file

Once you've `created your project directory` we need to include a keyword file which will supply our script with all our terms:

* `Create a .csv file` named `"keyword_list.csv"`
*  Cell A1 should include "Keywords"
*  All subsequent cells in column A should include your keywords


# Step 2: Getting things started in python


## Install Pytrends 

#### What is Pytrends? 

Pytrends is the unofficial API (Application Programming Interface) for google trends in Python. This is a simple API that allows you to track the different trends going on in the world’s most popular search engine – Google. It logs in into google on your behalf and takes in data at a much higher rate than manually possible. However, this particular API will be functional only for the current Google backend technology. Once that is changed this API shall no longer hold good. 

See: https://pypi.org/project/pytrends/#installation



#### To install Pytrends package with conda run:

In Jupyter Notebook you can execute Terminal commands in the notebook cells by prepending an exclamation point/bang(!) to the beginning of the command. This can be useful for many things such as getting information without having to open a Terminal/Command Prompt, or installing a conda package you are trying to use.

Be sure to provide "-y" to specify yes to the install prompt as you can not submit input to the commands when running.


In [11]:
from pytrends.request import TrendReq

## Import other useful packages 

* `pandas`: powerful data analysis tool (typically designed to handle `DataFrame` objects).
* `time`: a module provides various time-related functions.
* `os.path`: a module providing a big range of useful methods to manipulate files and directories.


In [5]:
import pandas as pd
from os import path
import time

## Define path to project directory. 

Needed only if `keyword_list.csv` is not in the same folder as your python code, or if you want to save the output `search_trends.csv` in another folder. 

In [29]:
data_path = "/home/mathilde/Dropbox/Cours et Tds/BigData&Health/DataChallenge"

# Step 2: Using pytrends

## Mock example

In [18]:
#Mock example

keywords=['corona','covid','symptomes']
trends = TrendReq(geo="FR")
trends.build_payload(keywords, timeframe='all')
print(trends.interest_over_time())

            corona  covid  symptomes isPartial
date                                          
2004-01-01       0      0          0     False
2004-02-01       1      0          0     False
2004-03-01       1      0          0     False
2004-04-01       1      0          0     False
2004-05-01       1      0          0     False
...            ...    ...        ...       ...
2021-09-01       2     64          0     False
2021-10-01       1     36          0     False
2021-11-01       1     38          0     False
2021-12-01       1     71          0     False
2022-01-01       1    100          0      True

[217 rows x 4 columns]


## Define Keywords 

In [13]:
colnames = ["keywords"]
# Function pd.read_csv reads a comma-separated values (csv) file into DataFrame.
df = pd.read_csv("keyword_list.csv", names=colnames)
df2 = df["keywords"].values.tolist()
df2.remove("Keywords")
print(df2)

['covid ', 'symptomes ', 'corona']


## Main code 

In [28]:
output_dataset = []
trends = TrendReq(geo="FR")

for x in range(0,len(df2)):
    keywords = [df2[x]]
    trends.build_payload(
    kw_list=keywords,
    cat=0,
    timeframe='2020-05-18 2022-02-06', # define time frame: from May 18, 2020 (when health data become available) until most recent date
    geo='FR-B')
    data = trends.interest_over_time() # creates dataframe with google trends values for current keyword
    data = data.drop(labels=['isPartial'],axis='columns')
    output_dataset.append(data)

print(output_dataset) 
result = pd.concat(output_dataset, axis=1)
result.to_csv('search_trends.csv')

#we want to keep missing values (if no queries)

[            covid 
date              
2020-05-24      17
2020-05-31      14
2020-06-07      11
2020-06-14      12
2020-06-21      12
...            ...
2022-01-09      44
2022-01-16      49
2022-01-23      38
2022-01-30      33
2022-02-06      26

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          46
2020-05-31          50
2020-06-07          43
2020-06-14          36
2020-06-21          46
...                ...
2022-01-09          94
2022-01-16          66
2022-01-23          68
2022-01-30          65
2022-02-06          54

[90 rows x 1 columns],             corona
date              
2020-05-24      63
2020-05-31      49
2020-06-07      39
2020-06-14      45
2020-06-21      43
...            ...
2022-01-09       9
2022-01-16       7
2022-01-23      10
2022-01-30       8
2022-02-06       6

[90 rows x 1 columns]]


### Define regions

In [43]:
# Dictionary of google regions with id and name 

regions_id_google = {'Nord_Pas_de_Calais':'O',
            'Picardie':'S',
            'Haute_Normandie':'Q',
            'Ile_de_France':'J',
            'Corse':'H',
            'PACA':'U',
            'Languedoc_Roussillon':'K',
            'Midi_Pyrénées':'N',
            'Aquitaine':'B',
            'Rhône_Alpes':'V',
            'Auvergne':'C', 
            'Limousin':'L',
            'Poitou_Charente':'T',
            'Basse_Normandie':'P',
            'Bretagne':'E',
            'Pays_de_la_Loire':'R',
            'Alsace':'A',
            'Lorraine':'M',
            'Champagne_Ardennes':'G',
            'Centre_Val_de_Loire':'F',
            'Bourgogne':'D',
            'Franche_Comté':'I'}


    
# Create dataframe for each region

region_df = regions_id_google.copy()
for reg in regions_id_google.keys():
    region_df[reg]=pd.DataFrame()   
    


In [70]:
trends = TrendReq(geo="FR")
output_dataset=pd.DataFrame()  
i=0

for region in regions_id_google.values():

    #retrieve region name (useful later for descriptive analysis)
    region_name = list(regions_id_google)[i]
    print(region, region_name)
    print('FR-'+region)  
    temp = []
    
    for x in range(0,len(df2)):
        keywords = [df2[x]]
        print(keywords)
        trends.build_payload(
        kw_list=keywords,
        cat=0,
        timeframe='2020-05-18 2022-02-06', # define time frame: from May 18, 2020 (when health data become available) until most recent date
        geo='FR-'+region)
        data = trends.interest_over_time() # creates dataframe with google trends values for current keyword
        data = data.drop(labels=['isPartial'],axis='columns')  
        temp.append(data)


    region_df[reg]=pd.DataFrame (pd.concat(temp, axis=1))
    region_df[reg].insert(0, 'region_code', region) 
    region_df[reg].insert(1, 'region_name', region_name) 
    output_dataset=output_dataset.append(region_df[reg])
    print(output_dataset)
    i=i+1
    
output_dataset.to_csv('search_trends.csv')    
        
        

O Nord_Pas_de_Calais
FR-O
['covid ']
[            covid 
date              
2020-05-24      16
2020-05-31      13
2020-06-07      11
2020-06-14      12
2020-06-21      12
...            ...
2022-01-09      42
2022-01-16      54
2022-01-23      39
2022-01-30      31
2022-02-06      22

[90 rows x 1 columns]]
['symptomes ']
[            covid 
date              
2020-05-24      16
2020-05-31      13
2020-06-07      11
2020-06-14      12
2020-06-21      12
...            ...
2022-01-09      42
2022-01-16      54
2022-01-23      39
2022-01-30      31
2022-02-06      22

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          52
2020-05-31          54
2020-06-07          47
2020-06-14          50
2020-06-21          54
...                ...
2022-01-09          82
2022-01-16          92
2022-01-23          82
2022-01-30          64
2022-02-06          54

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      16
2020-05-31  

[            covid 
date              
2020-05-24      25
2020-05-31      21
2020-06-07      19
2020-06-14      19
2020-06-21      20
...            ...
2022-01-09      60
2022-01-16      62
2022-01-23      39
2022-01-30      31
2022-02-06      24

[90 rows x 1 columns]]
['symptomes ']
[            covid 
date              
2020-05-24      25
2020-05-31      21
2020-06-07      19
2020-06-14      19
2020-06-21      20
...            ...
2022-01-09      60
2022-01-16      62
2022-01-23      39
2022-01-30      31
2022-02-06      24

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          52
2020-05-31          51
2020-06-07          45
2020-06-14          43
2020-06-21          46
...                ...
2022-01-09          75
2022-01-16          74
2022-01-23          61
2022-01-30          50
2022-02-06          44

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      25
2020-05-31      21
2020-06-07      19
2020-06-14 

[            covid 
date              
2020-05-24      16
2020-05-31      15
2020-06-07      12
2020-06-14      13
2020-06-21      14
...            ...
2022-01-09      45
2022-01-16      50
2022-01-23      40
2022-01-30      31
2022-02-06      23

[90 rows x 1 columns]]
['symptomes ']
[            covid 
date              
2020-05-24      16
2020-05-31      15
2020-06-07      12
2020-06-14      13
2020-06-21      14
...            ...
2022-01-09      45
2022-01-16      50
2022-01-23      40
2022-01-30      31
2022-02-06      23

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          48
2020-05-31          41
2020-06-07          37
2020-06-14          37
2020-06-21          47
...                ...
2022-01-09          85
2022-01-16          84
2022-01-23          76
2022-01-30          55
2022-02-06          52

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      16
2020-05-31      15
2020-06-07      12
2020-06-14 

[            covid 
date              
2020-05-24      17
2020-05-31      14
2020-06-07      11
2020-06-14      12
2020-06-21      12
...            ...
2022-01-09      44
2022-01-16      49
2022-01-23      38
2022-01-30      33
2022-02-06      26

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          46
2020-05-31          50
2020-06-07          43
2020-06-14          36
2020-06-21          46
...                ...
2022-01-09          94
2022-01-16          66
2022-01-23          68
2022-01-30          65
2022-02-06          56

[90 rows x 1 columns],             corona
date              
2020-05-24      63
2020-05-31      49
2020-06-07      39
2020-06-14      45
2020-06-21      43
...            ...
2022-01-09       9
2022-01-16       7
2022-01-23      10
2022-01-30       8
2022-02-06       6

[90 rows x 1 columns]]
           region_code region_name  covid   symptomes   corona
date                                                          
2020-05-

[            covid 
date              
2020-05-24      17
2020-05-31      14
2020-06-07      12
2020-06-14      13
2020-06-21      14
...            ...
2022-01-09      47
2022-01-16      50
2022-01-23      40
2022-01-30      35
2022-02-06      29

[90 rows x 1 columns]]
['symptomes ']
[            covid 
date              
2020-05-24      17
2020-05-31      14
2020-06-07      12
2020-06-14      13
2020-06-21      14
...            ...
2022-01-09      47
2022-01-16      50
2022-01-23      40
2022-01-30      35
2022-02-06      29

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          43
2020-05-31          33
2020-06-07          24
2020-06-14          44
2020-06-21          41
...                ...
2022-01-09          52
2022-01-16          73
2022-01-23         100
2022-01-30          34
2022-02-06          51

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      17
2020-05-31      14
2020-06-07      12
2020-06-14 

[            covid 
date              
2020-05-24      16
2020-05-31      14
2020-06-07      11
2020-06-14      12
2020-06-21      13
...            ...
2022-01-09      49
2022-01-16      49
2022-01-23      42
2022-01-30      36
2022-02-06      24

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          33
2020-05-31          37
2020-06-07          37
2020-06-14          33
2020-06-21          39
...                ...
2022-01-09          79
2022-01-16          72
2022-01-23          64
2022-01-30          55
2022-02-06          44

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      16
2020-05-31      14
2020-06-07      11
2020-06-14      12
2020-06-21      13
...            ...
2022-01-09      49
2022-01-16      49
2022-01-23      42
2022-01-30      36
2022-02-06      24

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          33
2020-05-31          37
2020-06-07          37
2020-06

[            covid 
date              
2020-05-24      19
2020-05-31      15
2020-06-07      16
2020-06-14      15
2020-06-21      14
...            ...
2022-01-09      51
2022-01-16      55
2022-01-23      45
2022-01-30      39
2022-02-06      26

[90 rows x 1 columns]]
['symptomes ']
[            covid 
date              
2020-05-24      19
2020-05-31      15
2020-06-07      16
2020-06-14      15
2020-06-21      14
...            ...
2022-01-09      51
2022-01-16      55
2022-01-23      45
2022-01-30      39
2022-02-06      26

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          50
2020-05-31          53
2020-06-07          51
2020-06-14          65
2020-06-21          49
...                ...
2022-01-09          60
2022-01-16          76
2022-01-23          72
2022-01-30          62
2022-02-06          51

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      19
2020-05-31      15
2020-06-07      16
2020-06-14 

[            covid 
date              
2020-05-24      20
2020-05-31      17
2020-06-07      14
2020-06-14      14
2020-06-21      13
...            ...
2022-01-09      53
2022-01-16      61
2022-01-23      45
2022-01-30      39
2022-02-06      26

[90 rows x 1 columns]]
['symptomes ']
[            covid 
date              
2020-05-24      20
2020-05-31      17
2020-06-07      14
2020-06-14      14
2020-06-21      13
...            ...
2022-01-09      53
2022-01-16      61
2022-01-23      45
2022-01-30      39
2022-02-06      26

[90 rows x 1 columns],             symptomes 
date                  
2020-05-24          60
2020-05-31          56
2020-06-07          35
2020-06-14          40
2020-06-21          53
...                ...
2022-01-09          86
2022-01-16          87
2022-01-23          80
2022-01-30          77
2022-02-06          50

[90 rows x 1 columns]]
['corona']
[            covid 
date              
2020-05-24      20
2020-05-31      17
2020-06-07      14
2020-06-14 

# Step 4: Intepreting data

## A note of caution

The "Interest Over Time" API returns historical, indexed data for when the keyword was searched most as shown on Google Trends'. What you get is not the direct representation of the query's search volume over time. Google Trends shows instead the relative popularity of a search query.

See an explanation in this blog post: https://ahrefs.com/blog/how-to-use-google-trends-for-keyword-research/